In [1]:
import os
import json
import pickle
import random
import time
import datetime
import numpy as np
import pandas as pd
import string
import re
from sklearn.metrics import accuracy_score, classification_report, f1_score

# read original data 

In [3]:
filename = 'train.json'
json_file = open('train.json', encoding='utf-8') 
train_data = json.load(json_file)
filename = 'dev.json'
json_file = open('dev.json', encoding='utf-8') 
dev_data = json.load(json_file)

filename = 'test-unlabelled.json'
json_file = open(filename, encoding='utf-8') 
test_data = json.load(json_file)

filename = 'dev-baseline-r.json'
json_file = open(filename, encoding='utf-8') 
dev_baseline_data = json.load(json_file)

filename = 'test-output.json'
json_file = open(filename, encoding='utf-8') 
result_example = json.load(json_file)
json_file.close()

def extract_data(data):
    text_list = []
    label_list = []
    for event in data:
        text_list.append(data[event]['text'])
        if 'label' in data[event].keys():
            label_list.append(data[event]['label'])

    return text_list,label_list
train_text_list, train_label_list = extract_data(train_data)
dev_text_list, dev_label_list = extract_data(dev_data)
test_text_list, test_label_list = extract_data(test_data)


# import pickle

with open('preprocessed_train_text.pickle', 'rb') as handle:
    preprocessed_train_text = pickle.load(handle)
with open('bow_train_text.pickle', 'rb') as handle:
    bow_train_text = pickle.load(handle)
print(len(preprocessed_train_text))
with open('preprocessed_dev_text.pickle', 'rb') as handle:
    preprocessed_dev_text = pickle.load(handle)
with open('bow_dev_text.pickle', 'rb') as handle:
    bow_dev_text = pickle.load(handle)
print(len(preprocessed_dev_text))

with open('preprocessed_test_text.pickle', 'rb') as handle:
    preprocessed_test_text = pickle.load(handle)
with open('bow_test_text.pickle', 'rb') as handle:
    bow_test_text = pickle.load(handle)
print(len(preprocessed_test_text))

# import extra data

In [3]:
def get_extra_data(data_dir,num=50):
    extra_dev_data = []
    for event in sorted(os.listdir(data_dir))[:num]:
        story_file = os.path.join(data_dir,event)
        with open(story_file,encoding='utf-8') as st_file:
            data = st_file.read()
        extra_dev_data.append(data[:3000])
        #st_file.close()
    return extra_dev_data

cnn_data_dir = 'C:\\Users\\pierce\\Desktop\\nlp\\project\\cnn\\stories'
#dm_data_dir = 'C:\\Users\\pierce\\Desktop\\nlp\\project\\dailymail\\stories'
extra_dev_data = get_extra_data(cnn_data_dir, num=100)
#extra_dev_data_dm = get_extra_data(dm_data_dir)
#extra_dev_data = extra_dev_data_cnn + extra_dev_data_dm
dev_text_list = dev_text_list + extra_dev_data
dev_text_list = dev_text_list[:200]
print(len(dev_text_list))
dev_label_list = dev_label_list+[0 for i in range(100)]
dev_label_list = dev_label_list[:200]
print(len(dev_label_list))

200
200


##add extra data to dev
extra_dev_bow, extra_dev_text = preprocess_bow(extra_dev_data)
preprocessed_dev_text.extend(extra_dev_text)
dev_label_list.extend([0 for i in range(100)])
bow_dev_text.extend(extra_dev_bow)
print(len(preprocessed_dev_text))
print(len(dev_label_list))


In [4]:
#extra training data
cnn_data_dir = 'C:\\Users\\pierce\\Desktop\\nlp\\project\\cnn\\stories'
#dm_data_dir = 'C:\\Users\\pierce\\Desktop\\nlp\\project\\dailymail\\stories'
extra_train_data_cnn = get_extra_data(cnn_data_dir, num=3100)
#extra_train_data_dm = get_extra_data(dm_data_dir, num=1550)
extra_train_data = train_text_list + extra_train_data_cnn[100:]
train_text_list = extra_train_data[:(3000+len(train_data))]
train_label_list = train_label_list+[0 for i in range(3000)]
train_label_list = train_label_list[:(3000+len(train_data))]
print(len(train_text_list))
print(len(train_label_list))

4168
4168


with open('train_text_list.pickle', 'wb') as handle:
    pickle.dump(train_text_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('train_label_list.pickle', 'wb') as handle:
    pickle.dump(train_label_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('dev_text_list.pickle', 'wb') as handle:
    pickle.dump(dev_text_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('dev_label_list.pickle', 'wb') as handle:
    pickle.dump(dev_label_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_text_list.pickle', 'wb') as handle:
    pickle.dump(test_text_list, handle, protocol=pickle.HIGHEST_PROTOCOL)


#add to training 
extra_train_bow, extra_train_text = preprocess_bow(extra_train_data)
preprocessed_train_text.extend(extra_train_text)
train_label_list.extend([0 for i in range(len(extra_train_data))])
bow_train_text.extend(extra_train_bow)
print(len(preprocessed_train_text))
print(len(train_label_list))

# IMPORT processed data

In [4]:
import pickle
with open('preprocessed_train_text.pickle', 'rb') as handle:
    preprocessed_train_text = pickle.load(handle)
print(len(preprocessed_train_text))
with open('preprocessed_dev_text.pickle', 'rb') as handle:
    preprocessed_dev_text = pickle.load(handle)
print(len(preprocessed_dev_text))

with open('preprocessed_test_text.pickle', 'rb') as handle:
    preprocessed_test_text = pickle.load(handle)
print(len(preprocessed_test_text))


train_text_list = []
for doc in preprocessed_train_text:
    train_text_list.append(' '.join(doc))
    
dev_text_list = []
for doc in preprocessed_dev_text:
    dev_text_list.append(' '.join(doc))
    
test_text_list = []
for doc in preprocessed_test_text:
    test_text_list.append(' '.join(doc))
    
dev_label_list = dev_label_list+[0 for i in range(100)]
dev_label_list = dev_label_list[:200]
train_label_list = train_label_list+[0 for i in range(3000)]
train_label_list = train_label_list[:(3000+len(train_data))]

4168
200
1410


# preprocessing 

In [5]:
def pre_preprocessing(dev_text_list):
    processed_list = []
    for text in dev_text_list:
        text = re.sub('@[^\s]+','',text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r"http\S+", "",text)
        text = re.sub(r"cnn", "",text)
        text =  re.sub(r"[']", '', text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"there's", "there is", text)
        text = re.sub(r"We're", "We are", text)
        text = re.sub(r"That's", "That is", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"they're", "they are", text)
        text = re.sub(r"Can't", "Cannot", text)
        text = re.sub(r"wasn't", "was not", text)
        text = re.sub(r"don\x89Ûªt", "do not", text)
        text = re.sub(r"aren't", "are not", text)
        text = re.sub(r"isn't", "is not", text)
        text = re.sub(r"What's", "What is", text)
        text = re.sub(r"haven't", "have not", text)
        text = re.sub(r"hasn't", "has not", text)
        text = re.sub(r"There's", "There is", text)
        text = re.sub(r"He's", "He is", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"You're", "You are", text)
        text = re.sub(r"I'M", "I am", text)
        text = re.sub(r"shouldn't", "should not", text)
        text = re.sub(r"wouldn't", "would not", text)
        text = re.sub(r"i'm", "I am", text)
        text = re.sub(r"I'm", "I am", text)
        text = re.sub(r"Isn't", "is not", text)
        text = re.sub(r"Here's", "Here is", text)
        text = re.sub(r"you've", "you have", text)
        text = re.sub(r"we're", "we are", text)
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"we've", "we have", text)
        text = re.sub(r"who's", "who is", text)
        text = re.sub(r"y'all", "you all", text)
        text = re.sub(r"would've", "would have", text)
        text = re.sub(r"it'll", "it will", text)
        text = re.sub(r"we'll", "we will", text)
        text = re.sub(r"We've", "We have", text)
        text = re.sub(r"he'll", "he will", text)
        text = re.sub(r"Y'all", "You all", text)
        text = re.sub(r"Weren't", "Were not", text)
        text = re.sub(r"Didn't", "Did not", text)
        text = re.sub(r"they'll", "they will", text)
        text = re.sub(r"they'd", "they would", text)
        text = re.sub(r"DON'T", "DO NOT", text)
        text = re.sub(r"they've", "they have", text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"should've", "should have", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"we'd", "we would", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"weren't", "were not", text)
        text = re.sub(r"They're", "They are", text)
        text = re.sub(r"let's", "let us", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"you're", "you are", text)
        text = re.sub(r"i've", "I have", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"i'll", "I will", text)
        text = re.sub(r"doesn't", "does not", text)
        text = re.sub(r"i'd", "I would", text)
        text = re.sub(r"didn't", "did not", text)
        text = re.sub(r"ain't", "am not", text)
        text = re.sub(r"you'll", "you will", text)
        text = re.sub(r"I've", "I have", text)
        text = re.sub(r"Don't", "do not", text)
        text = re.sub(r"I'll", "I will", text)
        text = re.sub(r"I'd", "I would", text)
        text = re.sub(r"Let's", "Let us", text)
        text = re.sub(r"you'd", "You would", text)
        text = re.sub(r"It's", "It is", text)
        text = re.sub(r"Ain't", "am not", text)
        text = re.sub(r"Haven't", "Have not", text)
        text = re.sub(r"Could've", "Could have", text)
        text = re.sub(r"youve", "you have", text)  
        text = re.sub(r"don’t", "do not", text)
        text = re.sub(r"I’m", "I am", text)
        text = re.sub(r"you’ve", "you have", text)
        text = re.sub(r"it’s", "it is", text)
        text = re.sub(r"doesn’t", "does not", text)
        text = re.sub(r"It’s", "It is", text)
        text = re.sub(r"Here’s", "Here is", text)
        text = re.sub(r"I’ve", "I have", text)
        text = re.sub(r"can’t", "cannot", text)
        text = re.sub(r"wouldn’t", "would not", text)
        text = re.sub(r"That’s", "That is", text)
        text = re.sub(r"You’re", "You are", text)
        text = re.sub(r"Don’t", "Do not", text)
        text = re.sub(r"Can’t", "Cannot", text)
        text = re.sub(r"you’ll", "you will", text)
        text = re.sub(r"I’d", "I would", text)
        text = re.sub(r"don’t", "do not", text)   
        text = re.sub(r"there’s", "there is", text)

        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r"\n", " ", text)
        text = re.sub(r"“|”", "", text)
        text = re.sub(r"‘|’", "", text)

        text = re.sub(r"cnn", "",text)
        text = re.sub(r"CNN", "",text)
        text = text.replace(u'\xa0', u' ')
        text = text.replace(u'\u200b', '')
        text = text.replace(u'‐', '')
        text = text.replace(u'‑', '')
        text = text.replace(u'—', '')
        text = text.replace(u'–', '')
        text = text.replace(u'…', '')
        
    
        processed_list.append(text)
        
    return processed_list


processed_dev_list = pre_preprocessing(dev_text_list)
processed_train_list = pre_preprocessing(train_text_list)
processed_test_list = pre_preprocessing(test_text_list)

def get_split(text_list):
    splitted_list = []
    for text1 in text_list:
        l_total = []
        l_parcial = []
        if len(text1.split())//150 >0:
            n = len(text1.split())//150
        else: 
            n = 1
        for w in range(n):
            if w == 0:
                l_parcial = text1.split()[:200]
                l_total.append(" ".join(l_parcial))
            else:
                l_parcial = text1.split()[w*150:w*150 + 200]
                l_total.append(" ".join(l_parcial))
        splitted_list.append(l_total)
    return splitted_list

splitted_train_list = get_split(processed_train_list)
splitted_dev_list = get_split(processed_dev_list)
splitted_test_list = get_split(processed_test_list)

In [6]:

from transformers import BertTokenizer
import torch
####################
def get_bertid(sentences, maxlen=300):
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    input_ids = []
    attention_masks = []
    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        if len(tokens) < maxlen:
            tokens = tokens + ['[PAD]' for _ in range(maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:int(maxlen*0.3)] + tokens[-int(maxlen*0.7):] #head 80 tokens tail 220 tokens
            tokens = tokens[:maxlen-1] + ['[SEP]']

        tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
        tokens_ids_tensor = torch.tensor(tokens_ids)
        attn_mask = (tokens_ids_tensor != 0).long()

        input_ids.append(tokens_ids_tensor)
        attention_masks.append(attn_mask)

    input_ids = torch.stack(input_ids) #list of tensor to tensor
    attention_masks = torch.stack(attention_masks)
    return input_ids, attention_masks

In [7]:
train_input_ids, train_attention_masks = get_bertid(processed_train_list, maxlen=64)
dev_input_ids, dev_attention_masks = get_bertid(processed_dev_list, maxlen=64)
test_input_ids, test_attention_masks = get_bertid(processed_test_list, maxlen=64)

In [8]:
from torch.utils.data import TensorDataset

train_labels = torch.tensor(train_label_list)
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

dev_labels = torch.tensor(dev_label_list)
dev_dataset = TensorDataset(dev_input_ids, dev_attention_masks, dev_labels)

#test data no label
t_labels = [0 for i in range(len(test_input_ids))]
t_labels = torch.tensor(t_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, t_labels)

In [9]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
validation_dataloader = DataLoader(dev_dataset,batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size )

In [10]:
device = torch.device("cuda")
print('There are %d GPU(s) available.' % torch.cuda.device_count())
print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1050


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class BertForSequenceClassification(nn.Module):
  
    def __init__(self, num_labels=2):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        #nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits

In [11]:
import torch.nn as nn
import torch.optim as optim
from transformers import BertForSequenceClassification, AdamW, BertConfig
configuration = BertConfig()
#configuration.output_hidden_states=True
print(configuration)
#net = SentimentClassifier()
net = BertForSequenceClassification(configuration)  
#    output_attentions = False, # Whether the model returns attentions weights.
#    output_hidden_states = False, # Whether the model returns all hidden-states.
#)
net.cuda()
criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=5e-6, eps=1e-8)

In [12]:
def seq_clf_evaluate(net, validation_dataloader, gpu):
  net.eval()
  loss_list = []
  pred_label_list = []
  with torch.no_grad():
    for seq, attn_masks, labels in validation_dataloader:
      seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
      loss,logits = net(seq, attention_mask=attn_masks, labels=labels)
      prob = torch.sigmoid(logits.unsqueeze(-1)).cpu()
      pred_label = np.argmax(prob, axis=1).flatten().tolist()
      pred_label_list.extend(pred_label)
      labels = labels.cpu()
      loss_list.append(loss.cpu())
  print('validation result prediction:',pred_label_list)
  print(print(classification_report(dev_label_list,pred_label_list)))
  print('avg loss', np.average(loss_list))

def clf_evaluate(net,criterion, validation_dataloader, gpu):
  net.eval()
  pred_label_list = []
  mean_loss = 0
  with torch.no_grad():
    for seq, attn_masks, labels in validation_dataloader:
      seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
      logits = net(seq, attn_masks)
      mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
      prob = torch.sigmoid(logits).cpu()
      pred_label = (prob>0.5).long().flatten().tolist()
      pred_label_list.extend(pred_label)
      labels = labels.cpu().tolist()
  print('validation result prediction:',pred_label_list)
  print(print(classification_report(dev_label_list,pred_label_list)))
  print('mean loss', mean_loss)

In [13]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42
epochs = 3
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

training_stats = []
total_t0 = time.time()

#with torch.no_grad():
# For each epoch...
for epoch_i in range(epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_train_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    bmodel.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        #clear previously calculated gradients
        bmodel.zero_grad()  
        #Obtaining the logits from the model
        loss, logits = bmodel(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        #Computing loss
        total_train_loss += loss.item()
        #Backpropagating the gradients
        loss.backward()

        torch.nn.utils.clip_grad_norm_(bmodel.parameters(), 1.0)
        #Optimization step
        opti.step()
        #scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    seq_clf_evaluate(bmodel, validation_dataloader, device)
    #print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(epoch_i, dev_acc, dev_loss))
    #if dev_acc > best_acc:
    #    print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
    #    best_acc = dev_acc
     #   torch.save(bmodel.state_dict(), 'sstcls_{}.dat'.format(epoch_i))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...


..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 1.46 GiB already allocated; 0 bytes free; 1.53 GiB reserved in total by PyTorch)

# Bert for embedding

In [ ]:
#####################
#for embedding 
#####################
class TextNet(nn.Module):
  
    def __init__(self, output_dim):
        super(TextNet, self).__init__()
        #self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        #self.fc = nn.Linear(768, output_dim)
        #self.tanh = torch.nn.Tanh()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        output = self.bert(input_ids, token_type_ids, attention_mask)
        text_embeddings = output[0][:,0,:]
        #features = self.fc(text_embeddings)
        #features = self.tanh(features)

        return text_embeddings

In [ ]:
net = TextNet(80) # bert embedding model
net.cuda()


In [ ]:
#bert sequential embedding
def bert_embedding_features(train_dataloader):
        gpu = device
        net.eval()
        loss_list = []
        pred_label_list = []
        embedding_list = []
        with torch.no_grad():
          final_features = np.empty((0, 768))
          for seq, attn_masks, labels in train_dataloader:
              seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
              features = net(seq, attention_mask=attn_masks)
              features = features.cpu().numpy()
              final_features = np.append(final_features, features, axis=0)
          #bert_embedding = hidden_state[0].cpu()
          #embedding_list.append(bert_embedding)
        #embedding_result = torch.stack(embedding_list)
        return final_features
train_bert_feature = bert_embedding_features(train_dataloader)
dev_bert_feature = bert_embedding_features(validation_dataloader)
test_bert_feature = bert_embedding_features(test_dataloader)
print(train_bert_feature.shape)
print(dev_bert_feature.shape)
print(test_bert_feature.shape)


In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, classification_report, f1_score
def train_ocsvm(train_set_d2v,dev_set_d2v,nu=0.3):
    svm_clf = OneClassSVM(kernel="rbf", degree=1, gamma='auto', coef0=0.0,
                           tol=0.0000001, nu=nu, shrinking=True, cache_size=200, 
                           max_iter=-1)
    svm_clf.fit(train_set_d2v)
    dev_predict = svm_clf.predict(dev_set_d2v)
    dev_predict[dev_predict == -1] = 0
    print(accuracy_score(dev_label_list,dev_predict))
    print(f1_score(dev_label_list, dev_predict, labels=None, pos_label=1, average='binary'))
    print(classification_report(dev_label_list,dev_predict))
    return svm_clf 
#svm_clf = train_ocsvm(train_set_d2v,dev_set_d2v)

In [ ]:
svm_clf = train_ocsvm(train_bert_feature,dev_bert_feature,nu=0.12)